In [19]:
from ecc.curve import Curve25519
from ecc.key import gen_keypair
from ecc.cipher import ElGamal
import random
import time
import numpy as np

## Encryption

In [9]:
# Plaintext
plaintext = b"I am plaintext."
# Generate key pair
pri_key, pub_key = gen_keypair(Curve25519)
# Encrypt using ElGamal algorithm
cipher_elg = ElGamal(Curve25519)
C1, C2 = cipher_elg.encrypt(plaintext, pub_key)
# Decrypt
new_plaintext = cipher_elg.decrypt(pri_key, C1, C2)

print(new_plaintext == plaintext)

True


## ECDSA

In [10]:
M = cipher_elg.curve.encode_point(plaintext)
G = cipher_elg.curve.G

k = random.randint(1, cipher_elg.curve.n)
r = ((k*G).x)%cipher_elg.curve.n
s = ((M.x + r*pri_key) % cipher_elg.curve.n)*pow(k, -1, cipher_elg.curve.n)

u1 = (M.x*pow(s, -1, cipher_elg.curve.n)) % cipher_elg.curve.n
u2 = (r*pow(s, -1, cipher_elg.curve.n)) % cipher_elg.curve.n

print((u1*G + u2*pub_key).x % cipher_elg.curve.n == r)

True


### Forgery Signature

In [11]:
k1 = random.randint(1, cipher_elg.curve.n)
k2 = random.randint(1, cipher_elg.curve.n)

R1 = k1*G + k2*pub_key
s1 = (-pow(k2, -1, cipher_elg.curve.n)*R1.x) % cipher_elg.curve.n
m = (k1*s1) % cipher_elg.curve.n

m*G == R1.x*pub_key+s1*R1

True

## Helper Functions

In [12]:
def convert_to_b_list(m, N):
    bits = [0]*N
    n = m
    for i in range(N):
        if (n==0):
            break
        bits[N-1-i]=n%2
        n = n >> 1
    return bits

def convert_from_b_list(l):
    m = 0
    for i in range(len(l)):
        m = m*2 + l[i]
    
    return m

## Lagrangian Interpolation on GF

In [13]:
import itertools
from functools import reduce
from sympy import symbols, Dummy
from sympy.polys.domains import ZZ
from sympy.polys.galoistools import (gf_irreducible_p, gf_add, \
                                     gf_sub, gf_mul, gf_rem, gf_gcdex)
from sympy.ntheory.primetest import isprime

class GF():
    def __init__(self, p, n=1):
        p, n = int(p), int(n)
        if not isprime(p):
            raise ValueError("p must be a prime number, not %s" % p)
        if n <= 0:
            raise ValueError("n must be a positive integer, not %s" % n)
        self.p = p
        self.n = n
        if n == 1:
            self.reducing = [1, 0]
        else:
            for c in itertools.product(range(p), repeat=n):
              poly = (1, *c)
              if gf_irreducible_p(poly, p, ZZ):
                  self.reducing = poly
                  break

    def add(self, x, y):
        return gf_add(x, y, self.p, ZZ)

    def sub(self, x, y):
        return gf_sub(x, y, self.p, ZZ)

    def mul(self, x, y):
        return gf_rem(gf_mul(x, y, self.p, ZZ), self.reducing, self.p, ZZ)

    def inv(self, x):
        s, t, h = gf_gcdex(x, self.reducing, self.p, ZZ)
        return s

    def eval_poly(self, poly, point):
        val = []
        for c in poly:
            val = self.mul(val, point)
            val = self.add(val, c)
        return [int(e) for e in val]

class PolyRing():
    def __init__(self, field):
        self.K = field

    def add(self, p, q):
        s = [self.K.add(x, y) for x, y in \
             itertools.zip_longest(p[::-1], q[::-1], fillvalue=[])]
        return s[::-1]       

    def sub(self, p, q):
        s = [self.K.sub(x, y) for x, y in \
             itertools.zip_longest(p[::-1], q[::-1], fillvalue=[])]
        return s[::-1]     

    def mul(self, p, q):
        if len(p) < len(q):
            p, q = q, p
        s = [[]]
        for j, c in enumerate(q):
            s = self.add(s, [self.K.mul(b, c) for b in p] + \
                         [[]] * (len(q) - j - 1))
        return s

def interp_poly(X, Y, K):
    R = PolyRing(K)
    poly = [[]]
    for j, y in enumerate(Y):
        Xe = X[:j] + X[j+1:]
        numer = reduce(lambda p, q: R.mul(p, q), ([[1], K.sub([], x)] for x in Xe))
        denom = reduce(lambda x, y: K.mul(x, y), (K.sub(X[j], x) for x in Xe))
        poly = R.add(poly, R.mul(numer, [K.mul(y, K.inv(denom))]))
    return poly

## Experiments

In [14]:
N = 254
field = GF(2, N)

### Initiator

In [15]:
total = 20
for t in range(10, 18):
    start = time.time()
    X = [[]]*t
    Y = [[]]*t
    for i in range(t):
        k1 = random.randint(1, cipher_elg.curve.n)
        k2 = random.randint(1, cipher_elg.curve.n)

        R1 = k1*G + k2*pub_key
        s1 = (-pow(k2, -1, cipher_elg.curve.n)*R1.x) % cipher_elg.curve.n
        m = (k1*s1) % cipher_elg.curve.n
        
        X[i] = convert_to_b_list(i, N)
        Y[i] = convert_to_b_list(m, N)
    end = time.time()
    # print("%d\t%.3f" % (total - t, end-start))
    print(end-start)

0.49385952949523926
0.5443542003631592
0.5919926166534424
0.6458594799041748
0.6913547515869141
0.7375326156616211
0.792057991027832
0.8426032066345215


### Witness

#### Single Witness Computational Time

In [16]:
total = 20
for t in range(10, 18):
    X = [[]]*t
    Y = [[]]*t
    for i in range(t):
        k1 = random.randint(1, cipher_elg.curve.n)
        k2 = random.randint(1, cipher_elg.curve.n)

        R1 = k1*G + k2*pub_key
        s1 = (-pow(k2, -1, cipher_elg.curve.n)*R1.x) % cipher_elg.curve.n
        m = (k1*s1) % cipher_elg.curve.n
        
        X[i] = convert_to_b_list(i, N)
        Y[i] = convert_to_b_list(m, N)

    ## single witness
    start = time.time()
    
    intpoly = interp_poly(X, Y, field)
    singleX = convert_to_b_list(t+1, N)
    singleY = field.eval_poly(intpoly, singleX)
    m = convert_from_b_list(singleY)
    
    k = random.randint(1, cipher_elg.curve.n)
    R = k*G
    s = ((m - R.x*pri_key) % cipher_elg.curve.n)*pow(k, -1, cipher_elg.curve.n)
    end = time.time()
    # print("%d\t%.3f" % (total - t, end-start))
    print(end-start)

0.4186136722564697
0.513164758682251
0.607562780380249
0.7467138767242432
0.9000668525695801
1.095958948135376
1.2919669151306152
1.5610368251800537


#### Multiple Witness Vehicles Computational Time

In [18]:
total = 20
for t in range(10, 18):
    X = [[]]*t
    Y = [[]]*t
    Sig = [{}]*t
    for i in range(t):
        k1 = random.randint(1, cipher_elg.curve.n)
        k2 = random.randint(1, cipher_elg.curve.n)

        R1 = k1*G + k2*pub_key
        s1 = (-pow(k2, -1, cipher_elg.curve.n)*R1.x) % cipher_elg.curve.n
        m = (k1*s1) % cipher_elg.curve.n
        
        X[i] = convert_to_b_list(i, N)
        Y[i] = convert_to_b_list(m, N)
        Sig[i] = {"R": R1, "s": s1, "m": m}

    ## (total-t) witness
    intpoly = interp_poly(X, Y, field)
    
    newX = [[]]*(total-t)
    newY = [[]]*(total-t)
    newSig = [{}]*(total-t)
    for i in range(total-t):
        newX[i] = convert_to_b_list(t+i, N)
        newY[i] = field.eval_poly(intpoly, newX[i])
        m = convert_from_b_list(singleY)
        
        k = random.randint(1, cipher_elg.curve.n)
        R = k*G
        s = ((m - R.x*pri_key) % cipher_elg.curve.n)*pow(k, -1, cipher_elg.curve.n)

        newSig[i] = {"R": R, "s": s, "m": m}

    totalSig = Sig + newSig

    start = time.time()
    for sig in totalSig:
        if not sig["R"].x*pub_key+sig["s"]*sig["R"] == sig["m"]*G:
            print("Invalid signature!")
            break
    mid = time.time()
    intpoly = interp_poly(X, Y, field)
    for i in range(len(newX)):
        if not newY[i] == field.eval_poly(intpoly, newX[i]):
            print("Invalid Polynomial!")
            break
    end = time.time()
    # print("%d\t%.3f" % (total - t, end-start))
    print("%.5f\t%.5f\t%.5f" %(mid-start, end-mid, end-start))

1.73471	0.83534	2.57005
1.71416	0.92401	2.63816
1.68698	1.01837	2.70535
1.65258	1.11974	2.77232
1.64269	1.24236	2.88505
1.63163	1.37512	3.00676
1.58672	1.51870	3.10542
1.56098	1.70514	3.26612


In [22]:
vehicles = 5
for eta in [0.1, 0.2, 0.3, 0.4, 0.5]:
    total = int(np.ceil(vehicles/eta))
    t = total - vehicles
    print("====initiator====")
    start = time.time()
    X = [[]]*t
    Y = [[]]*t
    Sig = [{}]*t
    for i in range(t):
        k1 = random.randint(1, cipher_elg.curve.n)
        k2 = random.randint(1, cipher_elg.curve.n)

        R1 = k1*G + k2*pub_key
        s1 = (-pow(k2, -1, cipher_elg.curve.n)*R1.x) % cipher_elg.curve.n
        m = (k1*s1) % cipher_elg.curve.n
        
        X[i] = convert_to_b_list(i, N)
        Y[i] = convert_to_b_list(m, N)
        Sig[i] = {"R": R1, "s": s1, "m": m}
    end = time.time()
    print(end-start)
    print("====Witness====")
    ## single witness
    start = time.time()
    
    intpoly = interp_poly(X, Y, field)
    singleX = convert_to_b_list(t+1, N)
    singleY = field.eval_poly(intpoly, singleX)
    m = convert_from_b_list(singleY)
    
    k = random.randint(1, cipher_elg.curve.n)
    R = k*G
    s = ((m - R.x*pri_key) % cipher_elg.curve.n)*pow(k, -1, cipher_elg.curve.n)
    end = time.time()
    # print("%d\t%.3f" % (total - t, end-start))
    print(end-start)


    ## (total-t) witness
    # intpoly = interp_poly(X, Y, field)
    
    newX = [[]]*(total-t)
    newY = [[]]*(total-t)
    newSig = [{}]*(total-t)
    for i in range(total-t):
        newX[i] = convert_to_b_list(t+i, N)
        newY[i] = field.eval_poly(intpoly, newX[i])
        m = convert_from_b_list(singleY)
        
        k = random.randint(1, cipher_elg.curve.n)
        R = k*G
        s = ((m - R.x*pri_key) % cipher_elg.curve.n)*pow(k, -1, cipher_elg.curve.n)

        newSig[i] = {"R": R, "s": s, "m": m}

    totalSig = Sig + newSig
    print("====Verification====")
    start = time.time()
    for sig in totalSig:
        if not sig["R"].x*pub_key+sig["s"]*sig["R"] == sig["m"]*G:
            print("Invalid signature!")
            break
    mid = time.time()
    intpoly = interp_poly(X, Y, field)
    for i in range(len(newX)):
        if not newY[i] == field.eval_poly(intpoly, newX[i]):
            print("Invalid Polynomial!")
            break
    end = time.time()
    # print("%d\t%.3f" % (total - t, end-start))
    print("%.5f\t%.5f\t%.5f" %(mid-start, end-mid, end-start))

3.83665	59.11491	62.95156
1.94578	2.97070	4.91647
1.37996	0.81776	2.19772
1.07449	0.40617	1.48065
0.86019	0.21213	1.07232


In [48]:
start = time.time()
m = 5629769104512241916374898635197064978639931267288972497570802529622899491984
G = cipher_elg.curve.G

k = random.randint(1, cipher_elg.curve.n)
R = k*G
s = ((m - R.x*pri_key) % cipher_elg.curve.n)*pow(k, -1, cipher_elg.curve.n)
end = time.time()
print(end-start)

0.035753488540649414


In [41]:
m*G

Point(X=25925308374563577056337187563840106216851094811464287060394422874010956958108, Y=18408911787612780815544535466002952916973192127858523694264411514237170125450, Curve=Curve25519)

In [45]:
R.x*pub_key+s*R

Point(X=25925308374563577056337187563840106216851094811464287060394422874010956958108, Y=18408911787612780815544535466002952916973192127858523694264411514237170125450, Curve=Curve25519)

In [6]:
def sort_by_first_element(a):
    res = a

    for i in range(len(a)):
        for j in reversed(range(i)):
            b = res[j][0] > res[j+1][0]
            c0 = b*(res[j][0]-res[j+1][0])
            c1 = b*(res[j][1]-res[j+1][1])
            res[j][0], res[j+1][0] = res[j+1][0] + c0, res[j][0] - c0
            res[j][1], res[j+1][1] = res[j+1][1] + c1, res[j][1] - c1
    
    return res

In [7]:
sort_by_first_element([[2.2, 0], [1.1, 1], [3.3, 2]])

[[3.3, 2], [2.2, 0], [1.1, 1]]

In [4]:
[1, 2.3]

[1, 2.3]